# Import Libraires

In [ ]:
# import required packages
import glob
import time
import pandas as pd
# from xml.dom import minidom
from nltk import ngrams
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import keras
import os

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Load the Dataset

In [ ]:
df=pd.read_csv('sqli_datasets.csv',encoding='utf-16') # Load Dataset

# Train

In [ ]:
# vectorization of data
# Import the CountVectorizer class from scikit-learn
from sklearn.feature_extraction.text import CountVectorizer

# Create an instance of CountVectorizer
# min_df=2 means ignore terms that appear in less than 2 documents
# max_df=0.7 means ignore terms that appear in more than 70% of the documents
# max_features=4096 sets the maximum number of features (vocabulary size) to 4096
# stop_words=stopwords.words('english') removes common English stopwords
vectorizer = CountVectorizer(min_df=2, max_df=0.7, max_features=4096, stop_words=stopwords.words('english'))

# Perform text vectorization on the 'Sentence' column of the DataFrame
# df['Sentence'].values.astype('U') converts the text data to Unicode format
# fit_transform() fits the vectorizer to the data and transforms the text into a numeric matrix
# toarray() converts the sparse matrix to a dense array representation
data = vectorizer.fit_transform(df['Sentence'].values.astype('U')).toarray()


In [ ]:
data.shape
data.shape=(5604,64,64,1)

In [ ]:
X=data
y=df['Label']

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
# X: input features, y: corresponding labels
# test_size=0.2 means 20% of the data will be used for testing
# random_state=42 sets a specific random seed for reproducibility
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape the training data
# X_train.shape[0] returns the number of samples in the training set
# -1 is used to infer the number of columns based on the original shape
trainX = X_train.reshape(X_train.shape[0], -1)

# Reshape the testing data
# X_test.shape[0] returns the number of samples in the testing set
# -1 is used to infer the number of columns based on the original shape
testX = X_test.reshape(X_test.shape[0], -1)


In [ ]:
X_train.shape

(4483, 64, 64, 1)

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
model = tf.keras.models.Sequential([
    # Convolutional layer with 64 filters, each of size 3x3
    # Activation function used is ReLU
    # Input shape is (64, 64, 1) for a single-channel (grayscale) image
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(64,64,1)),
    # Max pooling layer with pool size 2x2
    tf.keras.layers.MaxPooling2D(2,2),
    # Convolutional layer with 128 filters, each of size 3x3
    # Activation function used is ReLU
    tf.keras.layers.Conv2D(128,(3,3), activation='relu'),
    # Max pooling layer with pool size 2x2
    tf.keras.layers.MaxPooling2D(2,2),
    # Convolutional layer with 256 filters, each of size 3x3
    # Activation function used is ReLU
    tf.keras.layers.Conv2D(256,(3,3), activation='relu'),
    # Max pooling layer with pool size 2x2
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten layer to convert the 2D feature maps to 1D feature vectors
    tf.keras.layers.Flatten(),
    # Dense (fully connected) layer with 256 units
    # Activation function used is ReLU
    tf.keras.layers.Dense(256, activation='relu'),
    # Dense (fully connected) layer with 64 units
    # Activation function used is ReLU
    tf.keras.layers.Dense(64, activation='relu'),
    # Output layer with 1 unit
    # Activation function used is sigmoid, suitable for binary classification
    tf.keras.layers.Dense(1, activation='sigmoid')
])


In [ ]:
model.compile(
    # Loss function used for binary classification
    loss='binary_crossentropy', 
    # Optimizer used to update the weights of the model during training
    optimizer='adam', 
    # List of metrics to evaluate the model's performance
    metrics=['accuracy']
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 62, 62, 64)        640       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 31, 31, 64)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 29, 29, 128)       73856     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 14, 14, 128)      0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 12, 12, 256)       295168    
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 6, 6, 256)       

In [ ]:
classifier_nn = model.fit(
    X_train,  # Training data features
    y_train,  # Training data labels
    epochs=10,  # Number of training epochs
    verbose=True,  # Print training progress during each epoch
    validation_data=(X_test, y_test),  # Validation data for evaluation during training
    batch_size=128  # Number of samples per gradient update
)


Epoch 1/10
36/36 [==============================] - 95s 3s/step - loss: 0.3157 - accuracy: 0.8474 - val_loss: 0.2338 - val_accuracy: 0.9045
Epoch 2/10
36/36 [==============================] - 90s 2s/step - loss: 0.1786 - accuracy: 0.9099 - val_loss: 0.1835 - val_accuracy: 0.9188
Epoch 3/10
36/36 [==============================] - 94s 3s/step - loss: 0.1492 - accuracy: 0.9219 - val_loss: 0.1517 - val_accuracy: 0.9251
Epoch 4/10
36/36 [==============================] - 90s 2s/step - loss: 0.1421 - accuracy: 0.9242 - val_loss: 0.1690 - val_accuracy: 0.9340
Epoch 5/10
36/36 [==============================] - 92s 3s/step - loss: 0.1169 - accuracy: 0.9362 - val_loss: 0.1391 - val_accuracy: 0.9367
Epoch 6/10
36/36 [==============================] - 90s 3s/step - loss: 0.0979 - accuracy: 0.9480 - val_loss: 0.1182 - val_accuracy: 0.9483
Epoch 7/10
36/36 [==============================] - 94s 3s/step - loss: 0.0927 - accuracy: 0.9505 - val_loss: 0.1159 - val_accuracy: 0.9447
Epoch 8/10
36/36 [==

In [ ]:
pred=model.predict(X_test) # User Trained Model to Predic Data

36/36 [==============================] - 5s 135ms/step


In [ ]:
for i in range(len(pred)):
    # Check if the predicted value at index i is greater than 0.5
    if pred[i] > 0.5:
        # If the predicted value is greater than 0.5, assign it as 1
        pred[i] = 1
    elif pred[i] <= 0.5:
        # If the predicted value is less than or equal to 0.5, assign it as 0
        pred[i] = 0

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,pred)

0.9500446030330062

In [ ]:
for i,j in zip(y_test,pred):
    print(i==j)

[ True]
[ True]
[ True]
[False]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[False]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[False]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[False]


In [ ]:
from keras.models import load_model
import pickle

# Save the trained model as 'model_cnn.pkl'
model.save('model_cnn.h5')

# Save the vectorizer object as 'vectorizer_cnn.pkl'
# 'wb' mode is used for writing in binary mode
with open('vectorizer_cnn', 'wb') as fin:
    # Use pickle to dump the vectorizer object into the file
    pickle.dump(vectorizer, fin)


In [ ]:
accuracy_score

<function sklearn.metrics._classification.accuracy_score(y_true, y_pred, *, normalize=True, sample_weight=None)>

# Prediction

In [ ]:
import keras
from keras.models import load_model
import pickle


mymodel = tf.keras.models.load_model('model_cnn.h5')
myvectorizer = pickle.load(open("vectorizer_cnn", 'rb'))

In [ ]:
def clean_data(input_val):
    replacements = {
        '\n': '',
        '%20': ' ',
        '=': ' = ',
        '((': ' (( ',
        '))': ' )) ',
        '(': ' ( ',
        ')': ' ) ',
        '1 ': 'numeric',
        ' 1': 'numeric',
        "'1 ": "'numeric ",
        " 1'": " numeric'",
        '1,': 'numeric,',
        ' 2 ': ' numeric ',
        ' 3 ': ' numeric ',
        ' 3--': ' numeric--',
        ' 4 ': ' numeric ',
        ' 5 ': ' numeric ',
        ' 6 ': ' numeric ',
        ' 7 ': ' numeric ',
        ' 8 ': ' numeric ',
        '1234': ' numeric ',
        '22': ' numeric ',
        ' 200 ': ' numeric ',
        '23 ': ' numeric ',
        '"1': '"numeric',
        '1"': '"numeric',
        '7659': 'numeric',
        ' 37 ': ' numeric ',
        ' 45 ': ' numeric '
    }

    for key, value in replacements.items():
        input_val = input_val.replace(key, value)

    return input_val


In [ ]:
def predict_sqli_attack(text):
    # Clean the input data
    input_val = clean_data(text)
    
    # Prepare the input for the model
    input_val = [input_val]
    input_val = myvectorizer.transform(input_val).toarray()
    input_val.shape = (1, 64, 64, 1)
    
    # Make predictions using the model
    result = mymodel.predict(input_val)

    if result > 0.5:
        # If the predicted result is greater than 0.5, it's likely a SQL injection
        return 1
    elif result <= 0.5:
        # If the predicted result is less than or equal to 0.5, it's likely safe
        return 0


In [ ]:
# check for sql injection cheat sheet

predict_sqli_attack("d")

1/1 [==============================] - 0s 40ms/step


1